In [36]:
import pandas as pd

# Read the file
file = pd.read_json('../raw_data.json')
series = pd.Series(file['data'])

# convert json to dataframe
df = pd.json_normalize(series)

# print dataframe
df

,author,date,post_id,url,vote_count,post_content
0,zarobk,2014-05-05 10:09,id1031091,https://4programmers.net/Forum/Kariera/233131-...,41,"<div class=""post-content""><div><p>W związku z ..."
1,somekind,2014-05-05 10:29,id1031100,https://4programmers.net/Forum/Kariera/233131-...,7,"<div class=""post-content""><div><p>Jeśli chcesz..."
2,sssss,2014-05-05 10:30,id1031101,https://4programmers.net/Forum/Kariera/233131-...,3,"<div class=""post-content""><div><p>wiek: 26 lat..."
3,fasadin,2014-05-05 12:34,id1031139,https://4programmers.net/Forum/Kariera/233131-...,14,"<div class=""post-content""><div><p>ONE HUNDRED ..."
4,MateuszS,2014-05-06 08:58,id1031326,https://4programmers.net/Forum/Kariera/233131-...,6,"<div class=""post-content""><div><p>Tyle co rodz..."
...,...,...,...,...,...,...
3763,i1792,"dziś, 08:35",id1906962,https://4programmers.net/Forum/Kariera/233131-...,6,"<div class=""post-content""><div><p>Wiek: 31<br>..."
3764,ProgramistaXk,48 minut temu,id1907098,https://4programmers.net/Forum/Kariera/233131-...,0,"<div class=""post-content""><div><p>Wiek: 28<br>..."
3765,mythflame,2023-04-12 13:51,id1904930,https://4programmers.net/Forum/Kariera/233131-...,0,"<div class=""post-content""><div><p>Wiek: 25<br>..."
3766,PanAssembler,2023-04-12 22:13,id1904979,https://4programmers.net/Forum/Kariera/233131-...,4,"<div class=""post-content""><div><p>Wiek: ~22.5<..."
